# Azure ML Workspace

## Importing libraries

In [1]:
from azureml.core import Workspace, Environment, Model
from azureml.core.authentication import InteractiveLoginAuthentication
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
import json
import pandas as pd

## Initialise workspace
Initialise a workspace using the values from config.json
If this file is not found, an exception occurs
When an exception is detected, a workspace using the defined details is created

In [2]:
# An authentication error started occurring with the Azure Portal, this seems to fix it
# Issue found at https://github.com/Azure/MachineLearningNotebooks/issues/131
auth = InteractiveLoginAuthentication(tenant_id="57af78f2-c87d-4466-b7bb-6b6cc99ed124")

try:
# Try to load workspace
    wspace = Workspace.from_config(auth=auth)
except Exception as ex:
# Create new workspace if existing is not found
    wspace = Workspace.create(name="classification-workspace",
                              subscription_id="11636c4c-f408-4202-980a-b738e97f4d7f",
                              resource_group="classification-resources",
                              create_resource_group=True,
                              location="uksouth")

# Write workspace config to a local file
    wspace.write_config()

print("Workspace successfully loaded")

Deploying StorageAccount with name classifistoraged5b06aa15.
Deploying AppInsights with name classifiinsightsa50135a9.
Deployed AppInsights with name classifiinsightsa50135a9. Took 2.51 seconds.
Deploying KeyVault with name classifikeyvaultfe9d05ee.
Deployed KeyVault with name classifikeyvaultfe9d05ee. Took 22.34 seconds.
Deployed StorageAccount with name classifistoraged5b06aa15. Took 28.79 seconds.
Deploying Workspace with name classification-workspace.
Deployed Workspace with name classification-workspace. Took 17.11 seconds.
Workspace successfully loaded


## Create an environment
Create an environment for the model to run on the Azure servers

In [3]:
# Load conda dependencies required for this environment.
# Register environment for the workspace
env = Environment("env")

conda_dep = CondaDependencies.create(pip_packages=["pandas == 1.1.3",
                                                   "azureml-core",
                                                   "azureml-defaults >= 1.0.45"],
                                     conda_packages=["scikit-learn == 0.23.2"])

env.python.conda_dependencies = conda_dep

env.register(workspace=wspace)

print("Environment Registered")

# Get the environment created from the workspace and create a file to verify it has been defined
my_env = Environment.get(workspace=wspace, name="env")
my_env.save_to_directory("./environ", overwrite=True)

print("Environment Saved")

inference_config = InferenceConfig(entry_script="score.py",
                                   environment=my_env)

Environment Registered
Environment Saved


## Register the models
Register the saved model created by "knn-model-tuning" and "svm-model-tuning" for use on Azure ML

In [4]:
knn_model = Model.register(workspace=wspace,
                       model_name="knn",
                       model_path="models/knn.pkl",
                       model_framework="ScikitLearn",
                       description="Tuned KNN for Genre Classification")

svm_model = Model.register(workspace=wspace,
                       model_name="svm",
                       model_path="models/svm.pkl",
                       model_framework="ScikitLearn",
                       description="Tuned SVM for Genre Classification")

Registering model knn
Registering model svm


## Set up Azure Container Instance
Define ACI Webservice

In [5]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1)

## Deploy the models
Deploy the models onto the defined ACI container using the set environment, inference config and workspace

In [6]:
service = Model.deploy(workspace=wspace,
                       name="music-genre-service",
                       models=[knn_model, svm_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True)

service.wait_for_deployment(show_output=True)
print(service.get_logs())

print(service.scoring_uri)

Running........................................................................................................................
Succeeded
ACI service creation operation finished, operation "Succeeded"
2021-04-21T20:44:56,260561600+00:00 - gunicorn/run 
2021-04-21T20:44:56,290990800+00:00 - iot-server/run 
2021-04-21T20:44:56,303262100+00:00 - rsyslog/run 
2021-04-21T20:44:56,310713300+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_bc84c8b1e6f800381723c1af65ff7e9c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_bc84c8b1e6f800381723c1af65ff7e9c/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_bc84c8b1e6f800381723c1af65ff7e9c/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_bc84c8b1e6f800381723c1af65ff7e9c/lib/libssl.so.1.0.0: no version information ava

## Test the web service
Test the web service has successfully deployed using test data

In [7]:
# Reading the test set from file
data_df = pd.read_csv("testing_data.csv")
data_json_str = data_df.to_json(orient="records")

response_pred, response_acc, response_auroc, response_f1 = service.run(json.dumps(data_json_str))

response_pred_df = pd.DataFrame(response_pred)

print(response_pred_df)
print(response_acc)
print(response_auroc)
print(response_f1)

    KNN Prediction SVM Prediction
0        Classical      Classical
1            Disco            Pop
2             Jazz        Country
3             Jazz          Blues
4            Metal          Metal
..             ...            ...
295        Country         HipHop
296          Disco         HipHop
297           Jazz          Blues
298        Country         HipHop
299         Reggae         HipHop

[300 rows x 2 columns]
{'KNN Accuracy': 0.3, 'SVM Accuracy': 0.3433333333333333}
{'KNN AUROC': 0.7266192790644005, 'SVM AUROC': 0.7860937136748556}
{'KNN F1': 0.3, 'SVM F1': 0.3433333333333333}


## Delete ACI
Delete the ACI instance to clean up
(This will also delete the endpoint)

In [8]:
# service.delete()